# Gaining insights into the Amazon product network

# Milestone 2 - Data collection and description

The second task is to intimately acquaint yourself with the data, preprocess it and complete all the necessary descriptive statistics tasks. We expect you to have a pipeline in place, fully documented in a notebook, and show us that you’ve advanced with your understanding of the project goals by updating its README description.

## Dataset description

**show** 
- That you can handle the data in its size.

** todo **
- describe the two files of the dataset
- state the purpose of our project and describe how the we use the data for our project
- state the size of the dataset, make reasoning on what resources may be needed
- To handle the data, we have decided to implement our algorithms with PySpark.

We use the Amazon dataset, which consists of two JSON files:
- *metadata.json*: contains information about the products. Specifically, for each product.. The size of the dataset is 
- *reviews.json*: contains the reviews associated to each product. In particular, for each reviews.. The size of the dataset is

##  Exploratory analysis

**show**
- That you understand what’s into the data (formats, distributions, missing values, correlations, etc.).
- That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook.


**todo**
- put a list of all the categories in the dataset with counting
- show the category trees

## Preliminary processing

** show **
- That you considered ways to enrich, filter, transform the data according to your needs.

**todo**
- we only focus on few categories, which may be meaningful for our project (electronics, tools, musical instruments)
- Being our project mainly focused on products, rather than reviews, we have decided to reduce the reviews dataset, keeping only the average rating of the products

## Conclusion

** show **
- That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.

# Reminder: Internal steps before milestone2

- Define the rules for creating the graph (i.e. the influence of each relation type).
- Devise an efficient algorithm for extracting cliques or highly connected subgraphs, and, possibly, merging them into clusters.
- Find useful insights into the structure of these clusters, apart from obvious ones (the best product in a cluster). For example:
-- Do people always choose the most cheap product among related products?
-- Conversely, does the best product cost more than the others?
-- Are the best products sold only by well-known brands?